In [8]:
import mysql.connector 
import pandas as pd
import requests

In [9]:

baseDeDonnees = mysql.connector.connect(host="localhost",user="root",password="Cathie*-/63", database="dataengineer")
cursor = baseDeDonnees.cursor()


In [10]:
df_table = pd.read_sql("SELECT * FROM address", baseDeDonnees)
df_table.head()

c:\Users\cathe\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,address_id,address,city,postal_code,latitude,longitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,43.1961603,5.6094513
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,48.2909051,4.0457894
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,48.3613399,-4.7667997
3,4,6 BD DES ETINES,LE COTEAU,42120,46.021659,4.0921204
4,5,35 AV DU 159EME RIA,BRIANCON,05100,44.8984037,6.6436313


In [13]:
#nettoyage - recherche valeurs nulles
df_table.isnull().sum()

address_id       0
address          0
city             0
postal_code      0
latitude       552
longitude      552
dtype: int64

In [15]:
df_table.reset_index(drop=True)
df_table.head()

,address_id,address,city,postal_code,latitude,longitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,43.1961603,5.6094513
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,48.2909051,4.0457894
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,48.3613399,-4.7667997
3,4,6 BD DES ETINES,LE COTEAU,42120,46.021659,4.0921204
4,5,35 AV DU 159EME RIA,BRIANCON,05100,44.8984037,6.6436313


In [17]:
#merge all columns for create a column name "complete adress"
df_table["adress_merge"] = df_table["address"] +" "+ df_table["postal_code"] + " " + df_table["city"] 

#merge columns "postal code" and "city" only which will use if column "adress" is uncomplete or notfound by API nominatim
df_table["adress_merge2"] = df_table["postal_code"] + " " + df_table["city"] 

df_table.head()

,address_id,address,city,postal_code,latitude,longitude,adress_merge,adress_merge2
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,43.1961603,5.6094513,318 CHE DE ROUMAGOUA 13600 LA CIOTAT,13600 LA CIOTAT
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,48.2909051,4.0457894,19 RUE DES DAMES 10300 SAINTE SAVINE,10300 SAINTE SAVINE
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,48.3613399,-4.7667997,22 RUE AMIRAL GUEPRATTE 29217 LE CONQUET,29217 LE CONQUET
3,4,6 BD DES ETINES,LE COTEAU,42120,46.021659,4.0921204,6 BD DES ETINES 42120 LE COTEAU,42120 LE COTEAU
4,5,35 AV DU 159EME RIA,BRIANCON,05100,44.8984037,6.6436313,35 AV DU 159EME RIA 05100 BRIANCON,05100 BRIANCON


In [19]:
#I have create a function for create url to use with nominatim

def API_address(postal_address):
    link_main = 'https://nominatim.openstreetmap.org/?q='
    address = str(postal_address)
    link_end = '&format=json&limit=1'

    link = link_main + address.replace(" ","+").replace(",","+").replace("\'","+")
    return (link + link_end)

In [21]:
# Test function API_address :
API_address('27 Rue de Pologne, 78100 Saint-Germain-en-Laye')

'https://nominatim.openstreetmap.org/?q=27+Rue+de+Pologne++78100+Saint-Germain-en-Laye&format=json&limit=1'

In [22]:
#Create loop for retrieve latitudes and longitudes with each adress ("adress_merge" or "adress_merge2")

latitudes = []
longitudes = []

for row in range(len(df_table["adress_merge"])):

    #address
    link = API_address(df_table["adress_merge"][row]) # jusque là tout va bien :)

    #request link
    request_link = requests.get(link).json()

    #add coordinates to list
    if request_link:
        lat = [(request_link[0]['lat'])]
        latitudes.append(lat) 
        
        lon = [(request_link[0]['lon'])]
        longitudes.append(lon)
    else:
        link2 = API_address(df_table["adress_merge2"][row])
        request_link2 = requests.get(link2).json()
        lat = [(request_link2[0]['lat'])]
        latitudes.append(lat) 
        lon = [(request_link2[0]['lon'])]    
        longitudes.append(lon) 


df_table["latitude"] = latitudes
df_table["longitude"] = longitudes



df_table.head()

,address_id,address,city,postal_code,latitude,longitude,adress_merge,adress_merge2
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,[43.1961603],[5.6094513],318 CHE DE ROUMAGOUA 13600 LA CIOTAT,13600 LA CIOTAT
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,[48.2909051],[4.0457894],19 RUE DES DAMES 10300 SAINTE SAVINE,10300 SAINTE SAVINE
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,[48.3613399],[-4.7667997],22 RUE AMIRAL GUEPRATTE 29217 LE CONQUET,29217 LE CONQUET
3,4,6 BD DES ETINES,LE COTEAU,42120,[46.021659],[4.0921204],6 BD DES ETINES 42120 LE COTEAU,42120 LE COTEAU
4,5,35 AV DU 159EME RIA,BRIANCON,05100,[44.8984037],[6.6436313],35 AV DU 159EME RIA 05100 BRIANCON,05100 BRIANCON


In [23]:
#Search null values, for check table
df_table.isnull().sum()

address_id       0
address          0
city             0
postal_code      0
latitude         0
longitude        0
adress_merge     0
adress_merge2    0
dtype: int64

In [29]:
#replace value type list by value type string

for i in range(len(df_table['latitude'])):
  df_table['latitude'][i] = df_table.iloc[i,4][0]
  df_table['longitude'][i] = df_table.iloc[i,5][0]

df_table.head()

C:\Users\cathe\AppData\Local\Temp\ipykernel_15292\1301110200.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table['latitude'][i] = df_table.iloc[i,4][0]
C:\Users\cathe\AppData\Local\Temp\ipykernel_15292\1301110200.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table['longitude'][i] = df_table.iloc[i,5][0]


,address_id,address,city,postal_code,latitude,longitude,adress_merge,adress_merge2
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,43.1961603,5.6094513,318 CHE DE ROUMAGOUA 13600 LA CIOTAT,13600 LA CIOTAT
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,48.2909051,4.0457894,19 RUE DES DAMES 10300 SAINTE SAVINE,10300 SAINTE SAVINE
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,48.3613399,-4.7667997,22 RUE AMIRAL GUEPRATTE 29217 LE CONQUET,29217 LE CONQUET
3,4,6 BD DES ETINES,LE COTEAU,42120,46.021659,4.0921204,6 BD DES ETINES 42120 LE COTEAU,42120 LE COTEAU
4,5,35 AV DU 159EME RIA,BRIANCON,05100,44.8984037,6.6436313,35 AV DU 159EME RIA 05100 BRIANCON,05100 BRIANCON


In [33]:
# the values are integrate in my database "dataengineer" on "address" table

for i in range(len(df_table['latitude'])) :

    add_values_lat_et_lon = "UPDATE address SET latitude = "+df_table['latitude'][i]+", longitude = "+df_table['longitude'][i]+ " WHERE address_id = "+str(df_table['address_id'][i])
    cursor.execute(add_values_lat_et_lon)

    baseDeDonnees.commit()

cursor.close()
baseDeDonnees.close()

